---
title: Summary and Discussion
exports:
  - format: pdf
    template: arxiv_two_column
    output: pdf_builds/main.pdf
bibliography:
  - references.bib
---

## Data

Discussion of data and which assumptions we make about it to justify our analysis.  

Dataset used: @USDOTFlightDelays2015.

From the project description: If you propose any statistical hypothesis/model in your analysis, discuss your justifications, considering how the data was acquired. We are not expecting you to spend a lot of time dealing with how to best model or describe the data. Try to identify a few scientifically interesting but simple questions you can address and then focus on writing the code for such an analysis.

## Airlines and Airports

## Months and Days

## Predictions

## Conclusion

## Author Contributions
Below, it is described what each team member did in this project.

#### Jakob Bjerre Eriksen
.

#### Keyla Jaylin Barcenas
.

#### Keval Darshan Amin
.

#### Noa Adriana Gonzalez
.

## Other

```{bibliography}
```